In [ ]:
!pip install keras
!pip install tensorflow
!pip install pytorch_pretrained_bert pytorch-nlp

     |████████████████████████████████| 133kB 8.4MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 133kB 27.0MB/s 
     |████████████████████████████████| 7.3MB 28.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
ERROR: botocore 1.20.15 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
%matplotlib inline
import sys
import itertools
import numpy as np
import random as rn
import matplotlib.pyplot as plt
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
from torchnlp.datasets import imdb_dataset
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score


In [ ]:

distBertCovid = []
bertResultCovid = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_split = 10
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768, 1)
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        linear_output = self.linear(pooled_output)
        return linear_output
def sigmoid(x):                                        
    return 1 / (1 + np.exp(-x))        
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()
BATCH_SIZE = 12
EPOCHS = 3
optimizer = Adam(bert_clf.parameters(), lr=3e-5)
loss_func = nn.BCEWithLogitsLoss().cuda()

df = pd.read_csv('Covid19.csv' , header = 0, usecols=[0,1], encoding='latin-1')

df = df.sample(frac=1).reset_index(drop=True)
batch_1 = df
Kfold = StratifiedKFold(n_splits=n_split, random_state=0).split(batch_1, batch_1['label'])

#train_data, test_data = train_test_split(batch_1,test_size=0.2)
#len(train_data), len(test_data)
#train_texts = tuple(train_data['text']);
#test_texts = tuple(test_data['text']);
#train_labels = tuple(train_data['label']);
#test_labels = tuple(test_data['label']);
#df.head
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bertY = []
Y = []
bertYProb = []
distilBertY = []
distilBertYProb = []
for n_fold, (train_index, test_index) in enumerate(Kfold):
    
    train_data = batch_1.loc[train_index]
    test_data = batch_1.loc[test_index]
    train_texts = tuple(train_data['text']);
    test_texts = tuple(test_data['text']);
    train_labels = tuple(train_data['label']);
    test_labels = tuple(test_data['label']);
    train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
    test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))
    train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
    test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
    train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
    test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
    train_y = np.array(train_labels) == 'pos'
    test_y = np.array(test_labels) == 'pos'
    train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)
    train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
    test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
    train_tokens_tensor = torch.tensor(train_tokens_ids)
    train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

    test_tokens_tensor = torch.tensor(test_tokens_ids)
    test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

    train_masks_tensor = torch.tensor(train_masks)
    test_masks_tensor = torch.tensor(test_masks)
    train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

    test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)
    losses = []
    steps = []
    step = 0
    for epoch_num in range(EPOCHS):
        bert_clf.train()
        train_loss = 0
        for step_num, batch_data in enumerate(train_dataloader):
            token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
            probas = bert_clf(token_ids, masks)
        
            batch_loss = loss_func(probas, labels)
            train_loss += batch_loss.item()
        
        
            bert_clf.zero_grad()
            batch_loss.backward()
        

            clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
            optimizer.step()
        
            clear_output(wait=True)
            print('Epoch: ', epoch_num + 1)
            print("{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))
            losses.append(batch_loss.item())
            steps.append(step)
            step += 1
    #str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
    bert_clf.eval()
    bert_predicted_y = []
    bert_predicted = []
    all_logits = []
    with torch.no_grad():
        for step_num, batch_data in enumerate(test_dataloader):
            token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
            probas = bert_clf(token_ids, masks)
            numpy_probas = probas.cpu().detach().numpy()
            bert_predicted_y += list(sigmoid(numpy_probas[:, 0])>0.5)
            bert_predicted += list(sigmoid(numpy_probas[:, 0]))
    bertResult = classification_report(test_y, bert_predicted_y)  
    bertY += list(bert_predicted_y)
    bertYProb += list(bert_predicted)
    Y += list(test_y) 

    print(bertResult)    


    train_dataset_for_distill = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
    train_dataloader_for_distill = DataLoader(train_dataset, batch_size=BATCH_SIZE)

    bert_clf.eval()
    train_logits = []
    with torch.no_grad():
        for step_num, batch_data in enumerate(train_dataloader_for_distill):

            token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

            logits = bert_clf(token_ids, masks)
            numpy_logits = logits.cpu().detach().numpy()
        
            train_logits.append(numpy_logits)
    train_logits = np.vstack(train_logits)   
    
    distilled_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LinearRegression()).fit(train_texts, train_logits)
    distilled_predicted_logits = distilled_model.predict(test_texts)
    distilBertY += list(sigmoid(distilled_predicted_logits[:, 0]) > 0.5)
    distilBertYProb += list(sigmoid(distilled_predicted_logits[:, 0]))
    distBertResult = classification_report(test_y, sigmoid(distilled_predicted_logits[:, 0]) > 0.5)    
    print(distBertResult)
    distBertCovid.append(distBertResult)
    bertResultCovid.append(bertResult)
    print(roc_auc_score(Y, distilBertYProb))
    print(roc_auc_score(Y, bertYProb))


Epoch:  3
147/147.66666666666666 loss: 0.0001357670854132094 
              precision    recall  f1-score   support

       False       1.00      1.00      1.00        81
        True       1.00      1.00      1.00       115

    accuracy                           1.00       196
   macro avg       1.00      1.00      1.00       196
weighted avg       1.00      1.00      1.00       196

              precision    recall  f1-score   support

       False       0.98      0.99      0.98        81
        True       0.99      0.98      0.99       115

    accuracy                           0.98       196
   macro avg       0.98      0.99      0.98       196
weighted avg       0.98      0.98      0.98       196

0.9934399914956947
0.9994461571170404


In [ ]:
print(distBertCovid)

['              precision    recall  f1-score   support\n\n       False       0.92      0.96      0.94        82\n        True       0.97      0.94      0.96       115\n\n    accuracy                           0.95       197\n   macro avg       0.95      0.95      0.95       197\nweighted avg       0.95      0.95      0.95       197\n', '              precision    recall  f1-score   support\n\n       False       0.93      0.96      0.95        82\n        True       0.97      0.95      0.96       115\n\n    accuracy                           0.95       197\n   macro avg       0.95      0.96      0.95       197\nweighted avg       0.95      0.95      0.95       197\n', '              precision    recall  f1-score   support\n\n       False       0.98      1.00      0.99        82\n        True       1.00      0.98      0.99       115\n\n    accuracy                           0.99       197\n   macro avg       0.99      0.99      0.99       197\nweighted avg       0.99      0.99      0.99

In [ ]:
print(bertResultCovid)

['              precision    recall  f1-score   support\n\n       False       0.94      0.98      0.96        82\n        True       0.98      0.96      0.97       115\n\n    accuracy                           0.96       197\n   macro avg       0.96      0.97      0.96       197\nweighted avg       0.97      0.96      0.96       197\n', '              precision    recall  f1-score   support\n\n       False       1.00      0.98      0.99        82\n        True       0.98      1.00      0.99       115\n\n    accuracy                           0.99       197\n   macro avg       0.99      0.99      0.99       197\nweighted avg       0.99      0.99      0.99       197\n', '              precision    recall  f1-score   support\n\n       False       1.00      1.00      1.00        82\n        True       1.00      1.00      1.00       115\n\n    accuracy                           1.00       197\n   macro avg       1.00      1.00      1.00       197\nweighted avg       1.00      1.00      1.00

In [ ]:
print(classification_report(Y, distilBertY,digits=3)) 
print(classification_report(Y, bertY,digits=3)) 
print(roc_auc_score(Y, distilBertYProb))
print(roc_auc_score(Y, bertYProb))

              precision    recall  f1-score   support

       False      0.961     0.972     0.967       818
        True      0.980     0.972     0.976      1150

    accuracy                          0.972      1968
   macro avg      0.971     0.972     0.971      1968
weighted avg      0.972     0.972     0.972      1968

              precision    recall  f1-score   support

       False      0.994     0.991     0.993       818
        True      0.994     0.996     0.995      1150

    accuracy                          0.994      1968
   macro avg      0.994     0.994     0.994      1968
weighted avg      0.994     0.994     0.994      1968

0.9934399914956947
0.9994461571170404


In [ ]:
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
lrResultCovid = []
treeResultCovid=[]
rfResultCovid = []
baseline_predicted1 = []
baseline_predicted2 = []
baseline_predicted3 = []
baseline_predicted1Prob = []
baseline_predicted2Prob = []
baseline_predicted3Prob = []
labels = []
df = pd.read_csv('Covid192.csv' , header = 0, usecols=[0,1], encoding='latin-1')
df = df.sample(frac=1).reset_index(drop=True)
batch_1 = df
n_split = 10
Kfold = StratifiedKFold(n_splits=n_split, random_state=1).split(batch_1, batch_1['label'])
for n_fold, (train_index, test_index) in enumerate(Kfold):
    
    train_data = batch_1.loc[train_index]
    test_data = batch_1.loc[test_index]
    train_texts = tuple(train_data['text']);
    test_texts = tuple(test_data['text']);
    train_labels = tuple(train_data['label']);
    test_labels = tuple(test_data['label']);
    labels.append(test_labels)
    baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)
    baseline_predicted1.append(baseline_model.predict(test_texts))
    baseline_predicted1Prob.append(baseline_model.predict_proba(test_texts)[:, 1])
    #lrResultCovid.append(classification_report(test_labels, baseline_predicted))
    baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), tree.DecisionTreeClassifier()).fit(train_texts, train_labels)
    baseline_predicted2.append(baseline_model.predict(test_texts))
    baseline_predicted2Prob.append(baseline_model.predict_proba(test_texts)[:, 1])
    #treeResultCovid.append(classification_report(test_labels, baseline_predicted))
    baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), RandomForestClassifier()).fit(train_texts, train_labels)
    baseline_predicted3.append(baseline_model.predict(test_texts))
    baseline_predicted3Prob.append(baseline_model.predict_proba(test_texts)[:, 1])
    #rfResultCovid.append(classification_report(test_labels, baseline_predicted))



In [ ]:
labels = np.concatenate(labels)
baseline_predicted1 = np.concatenate(baseline_predicted1)
baseline_predicted2 = np.concatenate(baseline_predicted2)
baseline_predicted3 = np.concatenate(baseline_predicted3)
baseline_predicted1Prob = np.concatenate(baseline_predicted1Prob)
baseline_predicted2Prob = np.concatenate(baseline_predicted2Prob)
baseline_predicted3Prob = np.concatenate(baseline_predicted3Prob)


In [ ]:
print(roc_auc_score(labels, baseline_predicted1Prob))
print(roc_auc_score(labels, baseline_predicted2Prob))
print(roc_auc_score(labels, baseline_predicted3Prob))

print(classification_report(labels, baseline_predicted1,digits=3))
print(classification_report(labels, baseline_predicted2,digits=3))
print(classification_report(labels, baseline_predicted3,digits=3))

0.9917401934729456
0.897010736685447
0.98654087381737
              precision    recall  f1-score   support

         neg      0.949     0.939     0.944       818
         pos      0.957     0.964     0.961      1150

    accuracy                          0.954      1968
   macro avg      0.953     0.952     0.952      1968
weighted avg      0.954     0.954     0.954      1968

              precision    recall  f1-score   support

         neg      0.872     0.886     0.879       818
         pos      0.918     0.908     0.913      1150

    accuracy                          0.899      1968
   macro avg      0.895     0.897     0.896      1968
weighted avg      0.899     0.899     0.899      1968

              precision    recall  f1-score   support

         neg      0.930     0.925     0.928       818
         pos      0.947     0.950     0.949      1150

    accuracy                          0.940      1968
   macro avg      0.939     0.938     0.938      1968
weighted avg      0.